In [1]:
#Installing necessary library
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.5 MB/s 
     |████████████████████████████████| 5.5 MB 51.1 MB/s 
     |████████████████████████████████| 1.3 MB 45.3 MB/s 
     |████████████████████████████████| 182 kB 60.6 MB/s 
     |████████████████████████████████| 7.6 MB 16.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2e8746de9b22e9eda46d705727d4158dfeb3e7e5e1e2db917d99c1fd53844367
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
#Importing necessary libraries
import re
import numpy as np #version 1.19.5
from sentence_transformers import SentenceTransformer
import torch
import warnings
warnings.filterwarnings("ignore")
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from torch.utils.data import DataLoader
import os

In [3]:
#Mounting Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#Opening the PreProcessed File for Fine tunning [2012 Preprocessed dataset]
with open("/content/drive/MyDrive/NLP/Assign3/input2.txt", encoding="utf8") as fp:
    data = fp.read()
data = data.splitlines()
print(len(data))

87477


In [9]:
sent1 = []
sent2 = []
for i in range(0,len(data)):
    if i%2==0:
        sent1.append(data[i])
    else:
        sent2.append(data[i])
print(len(sent1))

43739


In [10]:
# Preparing final Train Data
train_data = [InputExample(texts=[s1, s2]) for s1,s2 in zip(sent1,sent2)]

In [ ]:
# Applying model using CLS pooling and HyperTunning Parameters

model_name = 'sentence-transformers/all-mpnet-base-v2'
word_embedding_model = models.Transformer(model_name, max_seq_length=32)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

train_loss = losses.MultipleNegativesSymmetricRankingLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=4,
    show_progress_bar=True,
)

#Saving the model
model.save('output/stsb-mpnet-base-v2')

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/684 [00:00<?, ?it/s]

In [111]:
#Opening Testing Dataset for calculating Cosine Similarities

file = "/content/drive/MyDrive/NLP/Assign2/TestingDataset2016Merged.txt"
content = open(file, encoding="utf-8")
output_predict = content.read()

In [112]:
output_predict = output_predict.splitlines()
listOdd = []
listEven = []
i=0
while(True):
    listOdd.append(output_predict[i])
    i+=1
    listEven.append(output_predict[i])
    i+=1
    if(i>=len(output_predict)):
        break

In [114]:
# Computing Embeddings
sentence_embeddings1 = model.encode(listOdd)
sentence_embeddings2 = model.encode(listEven)

In [119]:
# Calculate cosine similarities
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
results = []
for i in range(len(sentence_embeddings1)):
    value = cosine(sentence_embeddings1[i],sentence_embeddings2[i])
    value *= 5
    results.append(round(value))

In [121]:
#Saving the result in File
with open('/content/drive/MyDrive/NLP/Assign2/Fine_sbert_all-mpnet-base-v2_ep3bz64', 'w', encoding='utf8') as f:
    for i in results:
        f.write(f"{i}\n")

## **Without Fine Tune**

In [96]:
#Applying Model with HyperTunning
sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [97]:
#Evalauting cosine similarities on the 2016 STS dataset (for without hypertune)
listOdd1 = listOdd.copy()
listEven1 = listEven.copy()

In [98]:
#Compute cosine-similarities using embeddings
sentence_embeddings1 = sbert_model.encode(listOdd1,show_progress_bar=True)
sentence_embeddings2 = sbert_model.encode(listEven1,show_progress_bar=True)

Batches:   0%|          | 0/287 [00:00<?, ?it/s]

In [100]:
#Storing the cosine similarities into a List
Cosineresult = []
for i in range(len(sentence_embeddings1)):
    sim = cosine(sentence_embeddings1[i], sentence_embeddings2[i])
    cosine = ((sim*5))
    Cosineresult.append(round(cosine))

In [101]:
# Storing and writing the cosine similarities into a file
with open('/content/drive/MyDrive/NLP/Assign3/WFT_sbert_all-mpnet-base-v2', 'w', encoding="utf8") as f:
    for i in result:
        f.write(f"{i}\n")